<a href="https://colab.research.google.com/github/Nisarg03/Tracing-people-with-Facial-Recognition-using-OCR/blob/main/Coursera_face_recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Required Libraries
import zipfile

from PIL import Image, ImageDraw
import pytesseract
import cv2 as cv
import numpy as np
import os

# loading the face detection classifier
face_cascade = cv.CascadeClassifier('readonly/haarcascade_frontalface_default.xml')
SCALE_FACTOR = 1.3
NEIGHBOURS = 5

display(face_cascade)

In [ ]:
def img_to_text(orignalimage):
    # Convert to gray scale
    img = orignalimage.convert('L')
    
    #extract the text
    text = pytesseract.image_to_string(img)
    
    
    # Filter Punctuations
    words = text.replace('-\n','').replace('\n',' ').replace('.','').replace(',','')
    
    # List in lowercase and remove blanks
    words = words.lower().split(' ')
    if len(words) > 0:
        if ('' in words):
            words.remove('')
    
    # only keep unique values
    words = list(dict.fromkeys(words))
    return(words)

def img_to_faces (pil_img):
    face_images=[]
    # Convert to OpenCV format
    open_cv_image = np.array(pil_img) 
    open_cv_image = open_cv_image[:, :, ::-1].copy() 
    
    # loading the face detection classifier
    faces = face_cascade.detectMultiScale(open_cv_image,SCALE_FACTOR,NEIGHBOURS)
    # Crop image to only the faces
    for x,y,w,h in faces:
        im = pil_img.crop((x,y,x+w,y+h))
        face_images.append(im)
    return (face_images)
def Search(word):
    
    faces_found = SearchNewsPaperData(word,newspaper)
    display("Searching for "+word)
    for item in faces_found:
        display("Results found in file {}".format(item))
        if len(faces_found[item]['faces']) > 0:
            display(MakeSingleImage(faces_found[item]['faces'],120))
        else:
            display("No faces found.")    
    return()

def SearchNewsPaperData(wordToFind,data):
    newspapers={}
    for item in data:
        # Get the words from the newspaper
        words = data[item]['words']
        # Get the faces.
        faces = data[item]['faces']
        f = find_Words(wordToFind,words)
        if f > 0:
            # Found a match, store in a dictionary.
            newspapers.update({item:{'faces':faces}})
    return (newspapers)

def MakeSingleImage(images,picture_size):
    # Determining the size of the image
    img_per_row = min(5,len(images)) #To have not more than 5 images per row
    total_width = picture_size * 5
    max_height = int(picture_size * np.ceil(len(images)/5))
    new_im = Image.new('RGB', (total_width, max_height))
    x_offset = 0
    y_offset = 0
    count = 0
    for im in images:
        # Resize the image as a square to the desired size.
        im_sized = resize(im,picture_size)
        # paste into the master image.
        new_im.paste(im_sized, (x_offset,y_offset))
        x_offset += picture_size
        count += 1
        if (count % 5) == 0: #To change line after 5th image
            y_offset += picture_size
            x_offset = 0
    return(new_im)

def find_Words(wordToSearch,words):
    # Exact match(Lion = Lion) or Partial Match(Lion subset of Lions).
    if (wordToSearch in words) or (wordToSearch in s for s in words):
        return (1)
    else:
        # No match.
        return(0)

def resize(pil_img,basewidth):
    wpercent = (basewidth / float(pil_img.size[0]))
    hsize = int((float(pil_img.size[1]) * float(wpercent)))
    return(pil_img.resize((basewidth, hsize), Image.ANTIALIAS))
#Zip File Initiation
zip_name = os.getcwd() + "/readonly/small_img.zip"
zip_file_var = zipfile.ZipFile(zip_name,mode = 'r')

#Initialize the Newspaper Dictionary
newspaper = {}
files = zip_file_var.namelist()

#Iterate through all of the files
for file in files:
    print(file)
    newspaper_img = Image.open(io.BytesIO(zip_file_var.read(file)))
    
    #Get Text from Image
    words = img_to_text(newspaper_img)
    
    #Get Faces from Image
    faces = img_to_faces(newspaper_img)
    
    newspaper.update({file:{'words':words,'faces':faces}})


In [ ]:
Search("christopher")

Search("mark")